In [1]:
from gensim.models import KeyedVectors
from huggingface_hub import hf_hub_download

model = KeyedVectors.load_word2vec_format(hf_hub_download(repo_id="Word2vec/german_model", filename="german.model"), binary=True, unicode_errors="ignore")


/Users/herbishtini/anaconda3/envs/whisperx/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
german.model: 100%|██████████| 738M/738M [01:49<00:00, 6.73MB/s] 


In [2]:
def embed_text(text, model):
    words = text.split()
    # Filter out words not in the vocabulary
    words_in_vocab = [word for word in words if word in model]
    if not words_in_vocab:
        return None  # If no words in vocabulary, return None or handle accordingly
    # Calculate the mean of word vectors
    embedding = sum(model[word] for word in words_in_vocab) / len(words_in_vocab)
    return embedding

text_to_embed = "This is an example text."
embedding = embed_text(text_to_embed, model)
print(embedding)


[-2.59018481e-01  4.34756167e-02 -2.66063958e-01  3.77931029e-01
  1.10136062e-01 -8.98484215e-02  1.89175121e-02  3.29087824e-01
 -1.28299698e-01  4.63936925e-02  1.45522133e-01 -1.03230573e-01
 -5.00932150e-02  3.42555165e-01 -9.81640220e-02 -2.55089164e-01
 -1.84677124e-01 -2.35522494e-01  3.56340498e-01 -2.13016674e-01
 -1.45755038e-01 -1.04607038e-01  1.16282605e-01  8.31777230e-02
 -8.15125927e-02  2.70997155e-02  7.89255440e-01 -1.59523740e-01
 -1.19426183e-01 -1.12738639e-01 -1.32015675e-01 -3.05022925e-01
  6.43652901e-02 -1.31135732e-02 -3.24149698e-01  2.76411444e-01
 -4.23725337e-01 -1.59848675e-01 -2.82887250e-01  8.75200257e-02
 -5.24234287e-02  7.39130145e-03 -4.61482167e-01  2.58865148e-01
 -2.51074824e-02 -5.24699509e-01  2.62916833e-01  1.53730974e-01
  1.84853122e-01 -1.34917855e-01  2.02824082e-02 -9.49798990e-03
  2.62115359e-01 -1.44494161e-01  1.35141417e-01  2.62386110e-02
  2.20654905e-01 -1.20826840e-01 -1.63558245e-01 -8.21780190e-02
  4.07639861e-01 -1.20738

In [4]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

# Example text
text = "This is an example text to embed."

# Tokenize and encode the text
inputs = tokenizer(text, return_tensors="pt")

# Forward pass through the BERT model
with torch.no_grad():
    outputs = model(**inputs)

# Extract the embeddings from the last hidden layer
last_hidden_states = outputs.last_hidden_state

# Mean pooling of the last hidden layer to get a single vector for the entire text
bert_embedding = last_hidden_states.mean(dim=1).squeeze().numpy()

# Display the BERT embedding
print(bert_embedding)

ImportError: cannot import name 'get_full_repo_name' from 'huggingface_hub' (/Users/herbishtini/anaconda3/envs/whisperx/lib/python3.10/site-packages/huggingface_hub/__init__.py)

In [44]:
# eigene Funtionen aus dem src-Ordner des übergeordneten Verzeichnis importieren
import sys, os

import pandas as pd
import numpy as np

LIBPATH = os.path.abspath('../src')
if not LIBPATH in sys.path: sys.path.insert(1, LIBPATH)
import src.server.feed as feed
import src.server.core as core
import src.server.db as db
from dotenv import dotenv_values
#
from gensim.models import Word2Vec
#
import importlib
#
importlib.reload(core)
importlib.reload(feed)
importlib.reload(db)

data_list = [
    { 'id': 1, 'text': "Hallo und herzlich willkommen!",
     'start': 0.089,
     "end": 1.63,
     'episode_id': 15740694059,
     'speaker': 'unknown'
     },
    {'id': 2, 'text': "In der heutigen Sendung geht es noch einmal um language model programming languages.",
     'start': 1.63,
     'end': 5.993,
     'episode_id': 15740694059,
     'speaker': 'unknown2'
     },
    {'id': 3, 'text': "Diesmal haben wir mit lookerbäurekäne und markfischer 2 Entwickler von LMQL",
     'start': 5.993,
     'end': 12.497,
     'episode_id': 15740694059,
     'speaker': 'unknown1'
     }
]
'''
transcript_parsed = {
    'id':  [obj["id"] for obj in data_list],
    'text':  [obj["text"] for obj in data_list],
    'start': [obj["start"] for obj in data_list],
    'end': [obj["end"] for obj in data_list],
    'episode_id': [obj["episode_id"] for obj in data_list],
    'speaker': [obj["speaker"] for obj in data_list]
}
'''
rng = np.random.default_rng(seed=19530)
db.main(len(data_list))

transcript_parsed = {
    'episode_id': [obj["episode_id"] for obj in data_list],
    'text': [obj["text"] for obj in data_list],
    'start': [obj["start"] for obj in data_list],
    'end': [obj["end"] for obj in data_list],
    'speaker': [obj["speaker"] for obj in data_list],
    'embeddings': rng.random((len(data_list), len(data_list)))
}

df = pd.DataFrame.from_dict(transcript_parsed, orient='index')
df = df.transpose()
print(transcript_parsed)
print(df)
db.insert("segment", df)


Create connection...

List connections:
[('default', <pymilvus.client.grpc_handler.GrpcHandler object at 0x2b39b16f0>)]
{'episode_id': [15740694059, 15740694059, 15740694059], 'text': ['Hallo und herzlich willkommen!', 'In der heutigen Sendung geht es noch einmal um language model programming languages.', 'Diesmal haben wir mit lookerbäurekäne und markfischer 2 Entwickler von LMQL'], 'start': [0.089, 1.63, 5.993], 'end': [1.63, 5.993, 12.497], 'speaker': ['unknown', 'unknown2', 'unknown1'], 'embeddings': array([[0.6378742 , 0.43925104, 0.13211584],
       [0.46866668, 0.74429647, 0.03190612],
       [0.31691246, 0.60253741, 0.90073872]])}
    episode_id                                               text  start  \
0  15740694059                     Hallo und herzlich willkommen!  0.089   
1  15740694059  In der heutigen Sendung geht es noch einmal um...   1.63   
2  15740694059  Diesmal haben wir mit lookerbäurekäne und mark...  5.993   

      end   speaker                            

In [75]:
rng.random((1, 1))

array([[0.08848347]])

In [42]:
config = dotenv_values("../.env")
podcasts = feed.search_podcast('Knowledge Science - Alles über KI, ML und NLP', config=config)
#podcasts

In [ ]:
podcast_id = podcasts['feeds'][0]['id']
podcast_id

In [ ]:
episodes = feed.get_episodes(podcast_id, config=config)

for episode in episodes[0:min(5, len(episodes))]:
    print(episode['enclosureUrl'])

In [ ]:
episode_id, episode_url= episodes[0]['id'], episodes[0]['enclosureUrl']
episodes[0]

In [ ]:
filename_audio = feed.download_episode(episode_url, episode_id=episode_id)

In [ ]:
(filename_transcript, transcript) = core.transcribe(episode_id, filename_audio, diarize=False) # , language='de'
transcript_parsed = transcript['parsed_transcript']

In [ ]:
import importlib
importlib.reload(db)
dim=4

podcast = pd.DataFrame([{
    'id': 1234,
    'title': 'Knowledge Science - Alles über KI, ML und NLP',
    'embeddings': rng.random((1, 4))[0],
    'description': 'Knowledge Science - Der Podcast über Künstliche Intelligenz im Allgemeinen und Natural Language Processing im Speziellen. Mittels KI Wissen entdecken, aufbereiten und nutzbar machen, dass ist die Idee hinter Knowledge Science. Durch Entmystifizierung der Künstlichen Intelligenz und vielen praktischen Interviews machen wir dieses Thema wöchentlich greifbar.',
    'author': 'Sigurd Schacht, Carsten Lanquillon',
    'image': 'https://storage.buzzsprout.com/variants/3y8i7tuyi2gq6bj2d3dvlorswfu7/5cfec01b44f3e29fae1fb88ade93fc4aecd05b192fbfbc2c2f1daa412b7c1921.jpg',
    'language': 'de',
    'url': 'https://feeds.buzzsprout.com/1687822.rss',
    'episodeCount': 106
}])

db.insert("podcast", podcast)

In [ ]:
rng.random((1, 4))[0]

In [ ]:
#
episode = pd.DataFrame([{
    'id': 15740694059,
    'title': 'Episode 107 - Red-Teaming & Jailbreaking',
    'description': 'p>In der aktuellen Sendung des neuen Jahres starten wir mit einem spannenden Thema zum Jailbreaking und Red-Teaming von Sprachmodellen. Bleiben Sie dran.<br/><br/>Vielen Dank auch an unseren Sponsor XL2:\xa0<br/><br/>XL2 ist ein Joint Venture von Audi und Capgemini,\xa0<br/>dass die digitale Transformation in der Automobilindustrie vorantreibt. Das Unternehmen\xa0erarbeitet innovative End-to-End-Prozesse und implementiert maßgeschneiderte IT-Lösungen für seine Kunden.</p>Support the show',
    'datePublishedPretty': 'January 13, 2024 1:00pm',
    'image': 'https://storage.buzzsprout.com/variants/8zwtzsw47ln492vgbhgbmszq7reu/60854458c4d1acdf4e1c2f79c4137142d85d78e379bdafbd69bd34c85f5819ad.jpg',
    'enclosureUrl': 'https://www.buzzsprout.com/1687822/14302036-episode-107-red-teaming-jailbreaking.mp3',
    'podcast_id': 1234,
    'embeddings': rng.random((1, dim))[0]
}])

db.insert("episode", episode)

In [ ]:
df_transcript_segments = pd.DataFrame()
df_transcript_segments['embeddings'] = rng.random((1, 4))
len(df_transcript_segments)

In [ ]:

'''
transcript_parsed = pd.DataFrame([
    {
        'episode_id': 15740694059,
        'id': 00,
        'embeddings': rng.random((1, dim))[0],
        'speaker': 'unknown', 'start': 0.089,
        'end': 1.63,
        'text': ' Hallo und herzlich willkommen!',
    },
    {
        'embeddings': rng.random((1, dim))[0],'speaker': 'unknown', 'start': 1.63, 'end': 5.993, 'text': 'In der heutigen Sendung geht es noch einmal um language model programming languages.', 'episode': 15740694059},
    {
        'embeddings': rng.random((1, dim))[0],
        'speaker': 'unknown', 'start': 5.993, 'end': 12.497,
        'text': 'Diesmal haben wir mit lookerbäurekäne und markfischer 2 Entwickler von LMQL der language model query language zu Gast.', 'episode': 15740694059},
    {'embeddings': rng.random((1, dim))[0],'speaker': 'unknown', 'start': 12.497, 'end': 19.361, 'text': 'Die uns spannende Einblicke in die Entstehungsgeschichte und Fähigkeiten von LMQL und mögliche Weiterentwicklung geben werden.', 'episode': 15740694059},
    {'embeddings': rng.random((1, dim))[0],'speaker': 'unknown', 'start': 19.361, 'end': 24.404, 'text': 'Auch diese Sendung wird von XZ2 dem Joint Venture von Audi und Cup Gemini gesponsat.', 'episode': 15740694059}
])

db.insert("segment", transcript_parsed)
'''


In [ ]:
db.get_collection_data('segment')

In [ ]:
#max_dimension = max(len(vector) for vector in vectors)
#padded_vectors = [np.pad(vector, (0, max_dimension - len(vector))) for vector in vectors]
